In [29]:
from landslide4sense.data import LandslideDataSet
import albumentations as A
import numpy as np

In [4]:
import omegaconf
cfg = omegaconf.OmegaConf.load("../conf/config.yaml")
cfg

{'data': {'dir': '${hydra:runtime.cwd}/data/', 'train_list': '${hydra:runtime.cwd}/data/train_split_train.txt', 'eval_lists_paths': ['${hydra:runtime.cwd}/data/train_split_train.txt', '${hydra:runtime.cwd}/data/train_split_val.txt'], 'eval_names': ['train', 'val'], 'test_list': '${hydra:runtime.cwd}/data/valid.txt'}, 'model': {'input_size': '128,128', 'num_classes': 2, 'module': 'landslide4sense.models', 'name': 'Unet'}, 'train': {'run_name': 'baseline_split', 'tags': ['val_split'], 'early_stopping': {'monitor': 'val_f1', 'mode': 'max', 'patience': 3, 'best_result': 0.5}, 'steps_per_epoch': 34, 'batch_size': 100, 'num_workers': 2, 'learning_rate': 0.001, 'num_steps': 10000, 'num_steps_stop': 10000, 'weight_decay': 0.0005, 'gpu_id': 0, 'snapshot_dir': '/content/drive/MyDrive/data_science_projects/landslide4sense-2022/models/', 'restore_from': '', 'seed': 42}}

In [7]:
dataset = LandslideDataSet(
    "../data/",
    "../data/train_split_train.txt",
    max_iters=cfg.train.num_steps_stop * cfg.train.batch_size,
    set="labeled",
)

In [11]:
img, label, shape, name = next(iter(dataset))

In [24]:
img.shape

(14, 128, 128)

In [43]:
aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Transpose(p=0.5),
    A.RandomSizedCrop(
        min_max_height=(100, 100), 
        height=128, 
        width=128, 
        p=0.5
    )
])

augmented_channels = []

for ch in range(img.shape[0]):
    channel_img = img[ch, ...]
    augmented = aug(image=channel_img, mask=label)
    augmented_img = augmented["image"]
    mask = augmented["mask"]
    augmented_channels.append(augmented_img)

augmented_channels = np.array(augmented_channels)[np.newaxis, ...]
augmented_img = np.concatenate(augmented_channels, axis=0)
augmented_img

(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)
(128, 128)


array([[[1.3612964 , 1.3612964 , 1.3612964 , ..., 1.5300908 ,
         1.5300908 , 1.5300908 ],
        [1.3587042 , 1.3587042 , 1.3587042 , ..., 1.5300908 ,
         1.5300908 , 1.5300908 ],
        [1.354816  , 1.3565779 , 1.3574383 , ..., 1.5300908 ,
         1.5300908 , 1.5300908 ],
        ...,
        [1.784729  , 1.8573105 , 1.9194176 , ..., 1.38541   ,
         1.3863595 , 1.3883036 ],
        [1.784729  , 1.8573105 , 1.9194179 , ..., 1.3825616 ,
         1.3833034 , 1.3848222 ],
        [1.784729  , 1.8573105 , 1.9194179 , ..., 1.3767292 ,
         1.3770456 , 1.3776937 ]],

       [[1.1877761 , 1.2044891 , 1.1844723 , ..., 1.195528  ,
         1.1727474 , 1.1504633 ],
        [1.215631  , 1.2123814 , 1.1977614 , ..., 1.1921622 ,
         1.1797546 , 1.1718189 ],
        [1.2386274 , 1.2193177 , 1.207089  , ..., 1.1821527 ,
         1.1766899 , 1.177865  ],
        ...,
        [1.6002886 , 1.4500604 , 1.3226446 , ..., 1.1017742 ,
         1.1090305 , 1.1103652 ],
        [1.3